# Tiling - Predicting - Mosaicing

##### Before the data can be used in the model, they have to tiled to 256x256


https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7

TODO
1) Tif lesen, georeferenzierten Punkt oben links für späteres Nutzen erstellen 
2) JPEG erstellen
3) tiling
4) Klassifizierung 
5) wieder zusammensetzten der Tiles 
6) Georeferenzierung wieder hinzufügen (maybe 5?))

In [2]:
import os
import sys
from PIL import Image
from osgeo import gdal
import numpy as np
import rasterio
from osgeo import osr
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS
import glob
import re
import pandas as pd

In [39]:
savedir = "E:/W_Katrin/Kigali/Raster/tiles"
filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"

## 1) Create Point for Georeferencing [upper left corner]

In [107]:
# get coordinates from raster
src = gdal.Open(filename)

ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform() # ulx, uly = upper left x and y values

# get projection from raster
wkt = src.GetProjection()
src_crs = osr.SpatialReference(wkt).GetAttrValue('AUTHORITY',1)
print("SRC_CRS:", src_crs)

# create Point 
point = Point(ulx, uly)
print("Point:", point)

#create new GeoDataFrame to save the point in
newdata = gpd.GeoDataFrame()
newdata['geometry'] = None
newdata.loc[0, 'geometry'] = point

# Set the GeoDataFrame's coordinate system to the same as the src raster 
newdata.crs = CRS.from_epsg(src_crs)
print("GeoDataFrame CRS:", newdata.crs)

#save GeoDataFrame
outfp = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_ReferencePointUL.shp"
newdata.to_file(outfp)

SRC_CRS: 32736
Point: POINT (173723.36172185623 9789230.701895246)
GeoDataFrame CRS: epsg:32736


## 2) Convert input - file (Tif) to JPEG
#### using gdal translate
https://gis.stackexchange.com/questions/42584/how-to-call-gdal-translate-from-python-code

In [1]:
!gdal_translate -of JPEG -b 1 -b 2 -b 3 -scale -co "JGW=YES" E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg

Input file size is 13162, 8267
0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 6: driver JPEG does not support creation option JGW
Warning 6: JPEG driver doesn't support data type Float32. Only eight bit byte bands supported.



In [ ]:
dst = gdal.Open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg")
dst.GetMetadata()
!gdalinfo E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg

## 3) TILING of input data
##### - Input raster has to be tiled to feed the model for prediction - 

In [5]:
# define tiling function 
def tiling(filename, savedir, target_size): 
    img = Image.open(filename)
    width, height = img.size
    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size
    frame_num = 1
    for col_i in range(0, width, w):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            len = 5
            padding='0'
            save_to= os.path.join(savedir, "testing_{col}_{row}.jpg".format(col=format(col_i, padding + '>'+str(len)), row=format(row_i, padding + '>'+str(len))))
            crop.save(save_to)


In [6]:
# run tiling
import warnings
savedir = "E:/W_Katrin/Kigali/Raster/tiles"
filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg"

warnings.simplefilter('ignore', Image.DecompressionBombWarning)

tiling(filename, savedir, target_size = (256,256))

## 4) Predict and save predicted masks 

In [ ]:
#Load the model
model_improved = keras.models.load_model("17_model_improved")

In [ ]:
# add here the folder path, where your  data is stored
directory = "E:/W_Katrin/Kigali/Raster/tiles"
files = glob.glob(os.path.join(directory,'*'))

pattern_img = '.*\.jpg'
# create a list of all images in that directory [after converting them to jpg with the code in Github README]
files_img = list(filter(re.compile(pattern_img).match,files))

files_img = sorted(files_img)

print('Image files: {}\n'.format(len(files_img)))

In [ ]:
# create a dataframe of the satellite images and their respective mask 
df_files = pd.DataFrame(columns=['img'])
df_files['img'] = files_img

In [ ]:
def make_test_gen(df_test,target_size,batch_size,seed):# this function generates augmented data[!not only training data!]
    # or better said: it generates a function which generates augmented data[generator function]
    
    # https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/
    # generate batches of tensor image data with real-time data augmentation 
    # batches of original training data are randomly transformed and then used for training (not the original data!)
  test_gen = keras.preprocessing.image.ImageDataGenerator()
    
    # read data from dataframe and perform powerful on-the-fly image augmentation with ImageDataGenerator
  test_generator = test_gen.flow_from_dataframe(
          dataframe=df_test,        
          x_col='img',
          target_size=target_size,
          batch_size=batch_size,
          seed = seed,
          class_mode=None,
          color_mode='rgb',
          shuffle=False)
  return test_generator

In [ ]:
test_image_gen = make_test_gen(df_files, batch_size = 1,target_size = (256, 256), seed = 0) #seed 0 to keep the Reihenfolge

In [ ]:
# Generate output predictions for the input samples
mask_predicted = model_improved.predict(test_image_gen, verbose = 1)

In [ ]:
#save predicted maps

savedir = "E:/W_Katrin/Kigali/Raster/tiles/predicted_m17_1"
for i in range(len(mask_predicted)): 
    mask_pred = np.squeeze(mask_predicted[i])
    #print(mask_pred)
    #print(mask_pred.dtype)
    mask_pred = (255*(mask_pred - np.min(mask_pred))/np.ptp(mask_pred)).astype("uint8")  
    #break
    padding = '0'
    length = 4
    save_to = os.path.join(savedir, "testing_{x}.jpg".format(x = format(i, padding + '>' + str(length))))
    imageio.imwrite(save_to, mask_pred)

In [ ]:
# define function to display the results

def predict_buildings(img, mask_pred):
    i=0
    img=img[i].astype(int)
    mask_pred=np.squeeze(mask_pred[i])
    mask_pred_rend=np.argmax(mask_pred, axis=-1)
    plt.figure(figsize=0.8*np.array([3*6.4, 4.8]))
    plt.subplot(131) # = plt.subplot(1,3,1), 1 row, 4 columns,  first subplot (from right to left, first top row)
    plt.imshow(img)
    plt.subplot(132)
    plt.imshow(mask_pred)
    plt.subplot(133)
    plt.imshow(mask_pred_rend)
    plt.show()
    
    return mask_pred, mask_pred_rend

In [ ]:
mask_predicted_, mask_predicted_rend_= predict_buildings(img_test, mask_predicted)

## 5) Mosaic - Put images back together

### 5.1) Mosaic preparation

In [13]:
# create a dataframe of the satellite image tiles and their respective masks
directory_img = 'E:/W_Katrin/Kigali/Raster/tiles'
files = glob.glob(os.path.join(directory_img,'*'))
pattern_img = '.*\.jpg$'
files_img = list(filter(re.compile(pattern_img).match,files))
files_img = sorted(files_img)

directory_mask = "E:/W_Katrin/Kigali/Raster/tiles/predicted_m17_1"
files_ = glob.glob(os.path.join(directory_mask,'*'))
pattern_mask = '.*\.jpg$'
files_mask = list(filter(re.compile(pattern_mask).match,files_))
files_mask = sorted(files_mask)

df_files = pd.DataFrame(columns=['img','mask', 'X', 'Y'])
df_files['img'] = files_img
df_files['mask'] = files_mask

# read the upper left corner coordinates from the img-filename and save it in X and Y
for i in range(df_files.shape[0]): 
    string = df_files["img"].iloc[i]
    string = string.split('/')[4].split('.')[0].split('_')
    x = string[1]
    y = string[2]
    df_files['X'].iloc[i] = x
    df_files['Y'].iloc[i] = y

In [14]:
# create worldfiles for every mask tile to add georeferencing to them 

def create_worldfiles(dataframe, res, point_path):
    point = gpd.read_file(point_path)
    x_coord = point["geometry"].x[0]
    y_coord = point["geometry"].y[0]
    for i in range(len(dataframe)): 
        directory = dataframe["mask"].iloc[i].split('\\')[0]
        mask_str = dataframe["mask"].iloc[i].split('\\')[1].split('.')[0]
        f = open(os.path.join(directory, mask_str + '.jgw'), "w")
        f.write('{pixel_size}\n'.format(pixel_size = res)) #pixel size x
        f.write('{pixel_size}\n'.format(pixel_size = res)) #pixel size y
        f.write('0.0\n') #rotation
        f.write('0.0\n') #rotation
        f.write('{easting}\n'.format(easting = (x_coord + (res*int(dataframe["X"].iloc[i]))))) #location x
        f.write('{northing}\n'.format(northing = (y_coord + (res*int(dataframe["Y"].iloc[i])))))#location y
        f.close()  

In [15]:
pt_file = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_ReferencePointUL.shp"
create_worldfiles(dataframe = df_files, res =  0.4963, point_path = pt_file)

### 5.2) Mosaicing

In [3]:
os.getcwd()
os.chdir('E:/W_Katrin/Kigali/Raster/tiles/predicted_m17_1')
os.getcwd()

'E:\\W_Katrin\\Kigali\\Raster\\tiles\\predicted_m17_1'

In [15]:
!gdalinfo

Usage: gdalinfo [--help-general] [-json] [-mm] [-stats | -approx_stats] [-hist] [-nogcp] [-nomd]
                [-norat] [-noct] [-nofl] [-checksum] [-proj4]
                [-listmdd] [-mdd domain|`all`] [-wkt_format WKT1|WKT2|...]*
                [-sd subdataset] [-oo NAME=VALUE]* [-if format]* datasetname



FAILURE: No datasource specified.


In [26]:
#with gdal merge? !no seperate!
import subprocess
files_list = glob.glob('*.jpg')
#files_list = files_list[:2]

with open('mask_files.txt', 'w') as f:
    for item in files_list:
        f.write("%s\n" % item)

#print(files_list)

files_string = " ".join(files_list)
gdal_merge_loc = "E:\W_Katrin\miniconda3\envs\tue_dl2\Lib\site-packages\GDAL-3.4.1-py3.10-win-amd64.egg-info\scripts\gdal_merge.py"
#cmd = python + gdal_merge_loc + "-init 255 -o mask_predicted_mosaic_2.jpg -of JPEG -v" + files_string

#os.system(cmd)

In [ ]:
!gdal_merge.py -o mask_predicted_mosaic_1.jpg -of JPEG + files_string

In [19]:
!python "E:\W_Katrin\miniconda3\envs\tue_dl2\Lib\site-packages\GDAL-3.4.1-py3.10-win-amd64.egg-info\scripts\gdal_merge.py" init 255 -o mask_predicted_mosaic_2.tif -of GTiff -v testing_0000.jpg testing_0001.jpg

ERROR 4: init: No such file or directory
ERROR 4: 255: No such file or directory
Traceback (most recent call last):
  File "E:\W_Katrin\miniconda3\envs\tue_dl2\Lib\site-packages\GDAL-3.4.1-py3.10-win-amd64.egg-info\scripts\gdal_merge.py", line 11, in <module>
    sys.exit(main(sys.argv))
  File "E:\W_Katrin\miniconda3\envs\tue_dl2\lib\site-packages\osgeo_utils\gdal_merge.py", line 463, in main
    ysize = int((lry - uly) / geotransform[5] + 0.5)
ZeroDivisionError: float division by zero


In [29]:
!gdalbuildvrt -input_file_list mask_files.txt merged.vrt

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0000.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0001.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0002.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0003.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0004.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0005.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0006.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0007.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0008.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0009.jpg
Warning 6: gdalbuildvrt does not support rotated geo transforms. Skipping testing_0010.jpg